In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, MobileNetV2, Xception, NASNetMobile, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.legacy import Adam  # Use the legacy optimizer
from sklearn.model_selection import train_test_split

# Genetic Algorithm Segmentation
def genetic_algorithm_segment(image, population_size=20, generations=10):
    def initialize_population(pop_size):
        return np.random.randint(0, 256, pop_size)
    
    def fitness_function(image, threshold):
        _, binary_img = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
        return cv2.countNonZero(binary_img)

    def selection(population, fitness_scores):
        sorted_idx = np.argsort(fitness_scores)
        return population[sorted_idx[-2:]]  # Select top 2
    
    def crossover(parents):
        crossover_point = np.random.randint(0, 8)
        child1 = (parents[0] & (255 << crossover_point)) | (parents[1] & (255 >> (8 - crossover_point)))
        child2 = (parents[1] & (255 << crossover_point)) | (parents[0] & (255 >> (8 - crossover_point)))
        return [child1, child2]

    def mutation(offspring):
        mutation_point = np.random.randint(0, 8)
        return [offspring[0] ^ (1 << mutation_point), offspring[1] ^ (1 << mutation_point)]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    population = initialize_population(population_size)
    for generation in range(generations):
        fitness_scores = [fitness_function(gray, threshold) for threshold in population]
        parents = selection(population, fitness_scores)
        offspring = crossover(parents)
        population = np.concatenate((parents, mutation(offspring)))
    
    best_threshold = population[np.argmax([fitness_function(gray, threshold) for threshold in population])]
    _, segmented = cv2.threshold(gray, best_threshold, 255, cv2.THRESH_BINARY)
    return segmented

# Grad-CAM Implementation
class GradCAM:
    def __init__(self, model, layerName):
        self.model = model
        self.layerName = layerName
        self.gradModel = tf.keras.models.Model(
            inputs=[model.inputs],
            outputs=[model.get_layer(layerName).output, model.output])

    def compute_heatmap(self, image, eps=1e-8):
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = self.gradModel(inputs)
            loss = predictions[:, 0]

        grads = tape.gradient(loss, convOutputs)
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_JET):
        heatmap = cv2.applyColorMap((heatmap * 255).astype("uint8"), colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        return output

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
val_path = "/Users/niteshyadav/Lyme_Disease/val"
test_path = "/Users/niteshyadav/Lyme_Disease/test"

# Function to segment images using Genetic Algorithm
def segment_images(input_folder, output_folder):
    for subdir, _, files in os.walk(input_folder):
        for file in files:
            img_path = os.path.join(subdir, file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            segmented_img = genetic_algorithm_segment(img)  # Segment the image using GA
            output_subdir = os.path.join(output_folder, os.path.basename(subdir))
            os.makedirs(output_subdir, exist_ok=True)
            cv2.imwrite(os.path.join(output_subdir, file), segmented_img)

# Segment train, val, and test images
segment_images(train_path, train_path + "_segmented")
segment_images(val_path, val_path + "_segmented")
segment_images(test_path, test_path + "_segmented")

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

# Function to build and compile CNN model
def build_model(architecture, input_shape=(224, 224, 3)):
    if architecture == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'NASNetMobile':
        base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unknown architecture")

    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models
architectures = ['VGG16', 'ResNet50', 'DenseNet121', 'MobileNetV2', 'Xception', 'NASNetMobile', 'EfficientNetB0']
for architecture in architectures:
    model = build_model(architecture)
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(f'{architecture}_best_model.h5', save_best_only=True)
    ]
    model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=callbacks)
    model.evaluate(test_generator)

    # Grad-CAM for model explainability
    grad_cam = GradCAM(model, 'block5_conv3')  # Replace 'block5_conv3' with the name of the last conv layer of the model
    for img_path in test_generator.filepaths:
        img = cv2.imread(img_path)
        img_preprocessed = cv2.resize(img, (224, 224))
        img_preprocessed = np.expand_dims(img_preprocessed, axis=0)
        heatmap = grad_cam.compute_heatmap(img_preprocessed)
        heatmap_img = grad_cam.overlay_heatmap(heatmap, img)
        cv2.imshow('Grad-CAM', heatmap_img)
        cv2.waitKey(0)

cv2.destroyAllWindows()

# Hyperparameter tuning code (not fully implemented, as it's a complex process involving libraries like Keras Tuner or Optuna)
# This example shows a simple grid search approach for learning rate tuning
for lr in [1e-4, 1e-5, 1e-6]:
    model = build_model('ResNet50')  # Example with ResNet50
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, MobileNetV2, Xception, NASNetMobile, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split


# Genetic Algorithm Segmentation
def genetic_algorithm_segment(image, population_size=20, generations=10):
    def initialize_population(pop_size):
        return np.random.randint(0, 256, pop_size)
    
    def fitness_function(image, threshold):
        _, binary_img = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
        return cv2.countNonZero(binary_img)

    def selection(population, fitness_scores):
        sorted_idx = np.argsort(fitness_scores)
        return population[sorted_idx[-2:]]  # Select top 2
    
    def crossover(parents):
        crossover_point = np.random.randint(0, 8)
        child1 = (parents[0] & (255 << crossover_point)) | (parents[1] & (255 >> (8 - crossover_point)))
        child2 = (parents[1] & (255 << crossover_point)) | (parents[0] & (255 >> (8 - crossover_point)))
        return [child1, child2]

    def mutation(offspring):
        mutation_point = np.random.randint(0, 8)
        return [offspring[0] ^ (1 << mutation_point), offspring[1] ^ (1 << mutation_point)]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    population = initialize_population(population_size)
    for generation in range(generations):
        fitness_scores = [fitness_function(gray, threshold) for threshold in population]
        parents = selection(population, fitness_scores)
        offspring = crossover(parents)
        population = np.concatenate((parents, mutation(offspring)))
    
    best_threshold = population[np.argmax([fitness_function(gray, threshold) for threshold in population])]
    _, segmented = cv2.threshold(gray, best_threshold, 255, cv2.THRESH_BINARY)
    return segmented

# Grad-CAM Implementation
class GradCAM:
    def __init__(self, model, layerName):
        self.model = model
        self.layerName = layerName
        self.gradModel = tf.keras.models.Model(
            inputs=[model.inputs],
            outputs=[model.get_layer(layerName).output, model.output])

    def compute_heatmap(self, image, eps=1e-8):
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = self.gradModel(inputs)
            loss = predictions[:, 0]

        grads = tape.gradient(loss, convOutputs)
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_JET):
        heatmap = cv2.applyColorMap((heatmap * 255).astype("uint8"), colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        return output

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
val_path = "/Users/niteshyadav/Lyme_Disease/val"
test_path = "/Users/niteshyadav/Lyme_Disease/test"

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_path + "_segmented", target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

# Model building and training
def build_model(architecture, input_shape=(224, 224, 3)):
    if architecture == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'NASNetMobile':
        base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=input_shape)
    elif architecture == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unknown architecture")

    # Freeze the base model
    base_model.trainable = False

    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training and evaluating models
architectures = ['VGG16', 'ResNet50', 'DenseNet121', 'MobileNetV2', 'Xception', 'NASNetMobile', 'EfficientNetB0']
for architecture in architectures:
    model = build_model(architecture)
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(f'{architecture}_best_model.h5', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
    ]
    model.fit(train_generator, validation_data=val_generator, epochs=20, callbacks=callbacks)
    model.evaluate(test_generator)

    # Grad-CAM for model explainability
    grad_cam = GradCAM(model, 'block5_conv3')  # Replace 'block5_conv3' with the name of the last conv layer of the model
    for img_path in test_generator.filepaths:
        img = cv2.imread(img_path)
        img_preprocessed = cv2.resize(img, (224, 224))
        img_preprocessed = np.expand_dims(img_preprocessed, axis=0)
        heatmap = grad_cam.compute_heatmap(img_preprocessed)
        heatmap_img = grad_cam.overlay_heatmap(heatmap, img)
        cv2.imshow('Grad-CAM', heatmap_img)
        cv2.waitKey(0)

cv2.destroyAllWindows()

# Hyperparameter tuning code (not fully implemented, as it's a complex process involving libraries like Keras Tuner or Optuna)
# This example shows a simple grid search approach for learning rate tuning
for lr in [1e-4, 1e-5, 1e-6]:
    model = build_model('ResNet50')  # Example with ResNet50
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_generator, validation_data=val_generator, epochs=10)


Found 4046 images belonging to 2 classes.
Found 506 images belonging to 2 classes.
Found 507 images belonging to 2 classes.
Epoch 1/20
127/127 [==============================] - 387s 3s/step - loss: 0.5123 - accuracy: 0.8087 - val_loss: 0.4809 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 2/20


/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


127/127 [==============================] - 392s 3s/step - loss: 0.4953 - accuracy: 0.8136 - val_loss: 0.4806 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 3/20
127/127 [==============================] - 402s 3s/step - loss: 0.4964 - accuracy: 0.8139 - val_loss: 0.4808 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 4/20
127/127 [==============================] - 394s 3s/step - loss: 0.4907 - accuracy: 0.8141 - val_loss: 0.4794 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 5/20
127/127 [==============================] - 390s 3s/step - loss: 0.4864 - accuracy: 0.8141 - val_loss: 0.4797 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 6/20
127/127 [==============================] - 400s 3s/step - loss: 0.4844 - accuracy: 0.8141 - val_loss: 0.4796 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 7/20
127/127 [==============================] - 383s 3s/step - loss: 0.4849 - accuracy: 0.8141 - val_loss: 0.4804 - val_accuracy: 0.8142 - lr: 1.0000e-04
Epoch 8/20
127/127 [==============================]